In [1]:
!pip install -q -U trax

     |████████████████████████████████| 471kB 5.4MB/s 
     |████████████████████████████████| 174kB 13.4MB/s 
     |████████████████████████████████| 2.6MB 15.4MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 1.3MB 52.3MB/s 
     |████████████████████████████████| 348kB 52.7MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 
     |████████████████████████████████| 3.6MB 48.2MB/s 
     |████████████████████████████████| 890kB 51.4MB/s 
     |████████████████████████████████| 2.9MB 47.6MB/s 


In [2]:
import trax
# Use the tensorflow-numpy backend.
trax.fastmath.set_backend('tensorflow-numpy')
print(trax.fastmath.backend_name())

tensorflow-numpy


In [3]:
# https://www.tensorflow.org/datasets/catalog/fashion_mnist
train_stream = trax.data.TFDS('fashion_mnist', keys=('image', 'label'), train=True)()
eval_stream = trax.data.TFDS('fashion_mnist', keys=('image', 'label'), train=False)()

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteG7JCTM/fashion_mnist-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteG7JCTM/fashion_mnist-test.tfrecord


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [59]:
train_data_pipeline = trax.data.Serial(
    trax.data.Shuffle(),
    trax.data.Batch(8),
    trax.data.AddLossWeights(),
)

train_batches_stream = train_data_pipeline(train_stream)

eval_data_pipeline = trax.data.Serial(
    trax.data.Batch(8),
    trax.data.AddLossWeights(),
)

eval_batches_stream = eval_data_pipeline(eval_stream)

In [60]:
example_batch = next(train_batches_stream)
print(f'batch shape (image, label) = {[x.shape for x in example_batch]}')

batch shape (image, label) = [(8, 28, 28, 1), (8,), (8,)]


In [83]:
from trax import layers as tl
from trax.models.resnet import Resnet50

def get_model(n_output_classes=10):
  model = tl.Serial(
      tl.ToFloat(),

      tl.Conv(32, (3, 3), (1, 1), 'SAME'),
      tl.LayerNorm(),
      tl.Relu(),
      tl.MaxPool(),

      tl.Conv(64, (3, 3), (1, 1), 'SAME'),
      tl.LayerNorm(),
      tl.Relu(),
      tl.MaxPool(),

      tl.Flatten(),
      tl.Dense(n_output_classes),
      tl.LogSoftmax(),
  )
  return model

In [84]:
from trax.supervised import training

train_task = training.TrainTask(
    labeled_data=train_batches_stream,
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.01),
    n_steps_per_checkpoint=100,
)

eval_task = training.EvalTask(
    labeled_data=eval_batches_stream,
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
    n_eval_batches=20,
)

In [85]:
import os

model = get_model()

training_loop = training.Loop(model, 
                              train_task, 
                              eval_tasks=[eval_task], 
                              output_dir='./cnn_model')

training_loop.run(1000)


Step      1: Total number of trainable weights: 451658
Step      1: Ran 1 train steps in 1.56 secs
Step      1: train CrossEntropyLoss |  2.06045055
Step      1: eval  CrossEntropyLoss |  177.95621605
Step      1: eval          Accuracy |  0.23125000

Step    100: Ran 99 train steps in 2.99 secs
Step    100: train CrossEntropyLoss |  30.65670776
Step    100: eval  CrossEntropyLoss |  5.53605672
Step    100: eval          Accuracy |  0.54375000

Step    200: Ran 100 train steps in 2.82 secs
Step    200: train CrossEntropyLoss |  1.93918920
Step    200: eval  CrossEntropyLoss |  0.73415423
Step    200: eval          Accuracy |  0.71250000

Step    300: Ran 100 train steps in 1.68 secs
Step    300: train CrossEntropyLoss |  0.64221692
Step    300: eval  CrossEntropyLoss |  0.67144935
Step    300: eval          Accuracy |  0.75000000

Step    400: Ran 100 train steps in 1.81 secs
Step    400: train CrossEntropyLoss |  0.60595691
Step    400: eval  CrossEntropyLoss |  0.49500983
Step    40